In [55]:
import ipywidgets as widgets
from IPython.display import clear_output, display, Markdown, Latex, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout, Button, Box, FloatProgress
from ipyfilechooser import FileChooser

import zipfile
import pickle
import pandas

import ktrain

import re


In [56]:
fl = None

In [57]:
style = "<style> \
        div.abstract{ \
                background-color: #FEFFF033;border-color: #F0F3BD; \
                border-left: 5px solid #f7f1e7;\
                padding: 0.5em; \
        } \
        .tooltip { \
          position: relative; \
          display: inline-block; \
          border-bottom: 1px dotted black; \
          z-index: 1; \
        } \
        .tooltip .tooltiptext { \
          visibility: hidden; \
          width: 120px; \
          background-color: black; \
          color: #fff; \
          text-align: center; \
          padding: 5px 0; \
          border-radius: 6px; \
          position: absolute; \
          z-index: 3; \
        } \
        .tooltip:hover .tooltiptext { \
          visibility: visible; \
        }\
        </style>"
       
HTML(style)

In [58]:
RESEARCH_ASPECTS = ["Background", "Purpose", "Method", "Finding_Contribution", "Other"]
COLORS = ['#247BA033','#FF165433','#F3FFBD33','#CC4BC233','#0B032D44']

In [59]:
class FileLoader:
    def __init__(self):
        self.uids = {}
        self.docs = {}
        self.topic_models = {}
        self.available_classifications = []
        self.available_aspects = []
        self.papers_dict = {}
        
    def aspect_dict_to_string(self, aspect_dict):
        whole_text_dict = {}
        whole_text = ''
        for sentence in aspect_dict:
            whole_text_dict[sentence[0]] = sentence[1]

        for key in sorted(whole_text_dict):
            whole_text += whole_text_dict[key] + ' '
        
        return whole_text
        
        
    def abstract_dict_to_string(self, abstract_dict, highlighted_aspect):
        whole_text_dict = {}
        whole_text = ''
        for aspect in RESEARCH_ASPECTS:
            for sentence in abstract_dict[aspect]:
                if(highlighted_aspect is not None):
                    if(aspect == highlighted_aspect):
                        whole_text_dict[sentence[0]] = '<strong>{}</strong>'.format(sentence[1])
                    else:
                        whole_text_dict[sentence[0]] = sentence[1]
                else:
                    whole_text_dict[sentence[0]] = sentence[1]

        for key in sorted(whole_text_dict):
            whole_text += whole_text_dict[key] + ' '
        
        return whole_text
        
    def load_topic_model(self, model, params, vectorizer):
        """
        Load saved TopicModel object
        Args:
            fname(str): base filename for all saved files
        """

        tm = ktrain.text.get_topic_model(n_topics=params['n_topics'],
                             n_features = params['n_features'],
                             verbose = params['verbose'])
        tm.model = model
        tm.vectorizer = vectorizer
        
        return tm
        
    
    def load_files(self,path):        
        with zipfile.ZipFile(path) as zf:
            #LOAD PAPERS_DICT
            with zf.open(zf.namelist()[0]) as papers_dict:
                self.papers_dict = pickle.load( papers_dict )
        
            #LOAD UID_DICT
            
                
            for i in range(1, len(zf.namelist())):
                full_path = zf.namelist()[i]
                
                slash_indices = [i for i, ltr in enumerate(full_path) if ltr == '/']
                if(len(slash_indices)>2):
                    if full_path[slash_indices[0]+1:slash_indices[1]] not in self.available_classifications:
                        self.available_classifications.append(full_path[slash_indices[0]+1:slash_indices[1]])

                    if full_path[slash_indices[1]+1:slash_indices[2]] not in self.available_aspects:
                        self.available_aspects.append(full_path[slash_indices[1]+1:slash_indices[2]])
                
            for classification in self.available_classifications:
                self.topic_models[classification] = {}
                self.docs[classification] = {}
                
                with zf.open('built_files/'+classification+'/uids.p') as uids:
                    self.uids[classification] = pickle.load( uids )
                
                for aspect in self.available_aspects:
                    files_path = 'built_files/'+classification+'/'+aspect+'/'
                    
                    with zf.open(files_path+'docs.p') as docs:
                        self.docs[classification][aspect] = pickle.load( docs )
                    
                    with zf.open(files_path+aspect+'.tm_model') as model_zip, zf.open(files_path+aspect+'.tm_params') as params_zip, zf.open(files_path+aspect+'.tm_vect') as vectorizer_zip:
                        self.topic_models[classification][aspect] = self.load_topic_model(pickle.load(model_zip), pickle.load(params_zip),pickle.load(vectorizer_zip))
                        self.topic_models[classification][aspect].build(self.docs[classification][aspect],threshold=0)
                        self.topic_models[classification][aspect].train_recommender()

                        #self.docs[classification][aspect] = self.topic_models[classification][aspect].filter(self.docs[classification][aspect])
                        #self.topic_models[classification][aspect].visualize_documents(texts=self.docs[classification][aspect])
            #s.find('/')

In [60]:
fl = FileLoader()  

In [61]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%')

In [62]:
def selected_file():
    fl_box.children[1].disabled = False
    
    
title = widgets.Label(value='Upload the zip file')

#fc = FileChooser('./')

fl_box = widgets.Box(
    layout = box_layout,
    children=[
        FileChooser('./'),
        widgets.Button(
            description='LOAD BUILT FILES',
            disabled=True,
            button_style='', 
            icon='gear'),
        widgets.FloatProgress(disabled=True, min=0, max=100) 
    ]
)

def pressed_build_files(btn):
    fl_box.children[2].disabled = False
    path = fl_box.children[0].selected_path+'/'+fl_box.children[0].selected_filename
    fl.load_files(path)


fl_box.children[0].register_callback(selected_file)
fl_box.children[1].on_click(pressed_build_files)

### Load ZIP file (CARREGAR ARQUIVO ANTES DAS CELULAS SEGUINTES) ### 

In [63]:
#topic_model.get_docsdisplay(fl_box)

In [64]:
display(fl_box)

Box(children=(FileChooser(path='.', filename='', title='HTML(value='', layout=Layout(display='none'))', show_h…

In [66]:
pandas.set_option('display.max_colwidth', 1000)
def return_topics_df(classification_bool, aspect):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    #fl.topic_models[classification][aspect].print_topics(show_counts=True)\
    detailed_topics = {}
    topic_model = fl.topic_models[classification][aspect]
    topic_model._check_build()
    topic_counts = sorted([ (k, topic_model.topics[k], len(v)) for k,v in topic_model.topic_dict.items()], 
                            key=lambda kv:kv[-1], reverse=True)
    for (idx, topic, count) in topic_counts:
        detailed_topics[idx] =  {"Topics words":topic, "Number of papers":count}
    
    return pandas.DataFrame.from_dict(detailed_topics, orient='index')

df = pandas.DataFrame()


box1 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Get topics for:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Checkbox(
            value=True,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(obj):
    clear_output(wait=True)
    display(box1)
    display(return_topics_df(box1.children[2].value, box1.children[1].value))
    
box1.children[3].on_click(btn_eventhandler)



#display(box)

In [67]:
def get_num_docs_topics(classification_bool, aspect, topic_id):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    topic_model = fl.topic_models[classification][aspect]
    return len(topic_model.get_docs(topic_ids=[topic_id]))

def get_selectable_topics(classification_bool, aspect, filter_words):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    selectable_topics = []
    topic_model = fl.topic_models[classification][aspect]
    topics = topic_model.get_topics(n_words=10, as_string=True)
    
    topic_model._check_build()
    topic_counts = sorted([ (k, topic_model.topics[k], len(v)) for k,v in topic_model.topic_dict.items()], 
                            key=lambda kv:kv[-1])
    for (idx, topic, count) in topic_counts:
        if(any(topic_word in filter_words.split() for topic_word in topic.split()) or filter_words == ''):
            selectable_topics.append((topic, idx))
         
    return selectable_topics

def return_docs_in_topic(classification_bool, aspect, topic_id, n):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    docs_in_topic = {}
    topic_model = fl.topic_models[classification][aspect]
    docs = topic_model.get_docs(topic_ids=[topic_id], rank=True)


    for i in range(0,n):
        try:
            doc_uid = fl.uids[classification][docs[i][1]]
            doc_title = fl.papers_dict[fl.uids[classification][docs[i][1]]]['title']
            publish_date = fl.papers_dict[fl.uids[classification][docs[i][1]]]['publish_time']
            doc_abstract = fl.abstract_dict_to_string(fl.papers_dict[fl.uids[classification][docs[i][1]]]['abstract'], aspect)
            if(len(doc_abstract) < 10):
                doc_abstract = doc_title
            
            docs_in_topic[doc_uid] ={'title':doc_title, 'publish_date':pandas.to_datetime(publish_date), 'abstract': doc_abstract, 'score':docs[i][2]}
        except Exception as e:
            break

    return docs_in_topic
    #for doc in docs:
        #docs_in_topic[doc[]]

pandas.set_option('display.max_colwidth', 1000)


items_layout = Layout( width='auto')

#pandas.set_option('display.max_colwidth', 40)

#topics_dropdown = widgets.Dropdown(
#    options=[('Whole abstract', topics_all_50_whole), ('Abstract Findings', topics_findings_50_whole), ('Abstract Purpose', topics_findings_50_whole)],
#    value=topics_findings_50_whole,
#    description='Get topics for:',
#)
df = pandas.DataFrame()


box2 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Get topics for:', layout={'width': 'max-content'}),
        widgets.Dropdown(
            options=fl.available_aspects,
            description='Aspect:'
        ),
        widgets.Dropdown(
            options=[],
            description='Topic:'
        ),
        widgets.Textarea(
            value='',
            placeholder='Query Sentence',
            description='Filter topic:',
            disabled=False
        ),
        widgets.Checkbox(
            value=True,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        widgets.IntSlider(
            value=7,
            min=0,
            max=10,
            step=1,
            description='Number of documents:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            style={'description_width': 'initial'}
        ),
        
        widgets.RadioButtons(
            options=[('score', None), ('date published', 'publish_date')],
        #    value='pineapple', # Defaults to 'pineapple'
        #    layout={'width': 'max-content'}, # If the items' names are long
            description='Sort by:',
        ),
        
        widgets.Button(
            description='Search',
            disabled=True,
        ),
        
    ]
)

def select_eventhandler(obj):
    box2.children[7].disabled = True
    box2.children[2].options = get_selectable_topics(box2.children[4].value, box2.children[1].value, box2.children[3].value)
    box2.children[5].max = get_num_docs_topics(box2.children[4].value, box2.children[1].value, box2.children[2].value)
    box2.children[7].disabled = False
    
def checkbox_eventhandler(obj):
    box2.children[7].disabled = True
    box2.children[2].options = get_selectable_topics(box2.children[4].value, box2.children[1].value, box2.children[3].value)
    box2.children[5].max = get_num_docs_topics(box2.children[4].value, box2.children[1].value, box2.children[2].value)
    box2.children[7].disabled = False
    
def search_btn_eventhandler(btn):
    clear_output(wait=True)
    display(box2)
    
    docs_in_topic = return_docs_in_topic(box2.children[4].value, box2.children[1].value, box2.children[2].value, box2.children[5].value)
    
    df = pandas.DataFrame.from_dict(docs_in_topic, orient='index')
    
    if(box2.children[6].value is not None):
        df = df.sort_values(by=[box2.children[6].value], ascending=False)
    
        
    display(Markdown(df.to_markdown()))

    
def topic_eventhandler(btn):
    box2.children[5].max = get_num_docs_topics(box2.children[4].value, box2.children[1].value, box2.children[2].value)
    
def filter_eventhandler(text_field):
    box2.children[2].options = get_selectable_topics(box2.children[4].value, box2.children[1].value, box2.children[3].value)
    
select_eventhandler(box2.children[1])
box2.children[1].observe(select_eventhandler, names=['value'])
box2.children[4].observe(checkbox_eventhandler, names=['value'])
box2.children[2].observe(topic_eventhandler, names=['value'])
box2.children[3].observe(filter_eventhandler, names=['value'])
box2.children[7].on_click(search_btn_eventhandler)


#display(box2)

In [80]:
def get_query_results(classification_bool, aspect, query_text):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    topic_model = fl.topic_models[classification][aspect]
        
    results = []
    final_results = {}
            
    for i, doc in enumerate(topic_model.recommend(text=query_text, n=50, n_neighbors=100)):
        uid = fl.uids[classification][doc[1]]
        abstract_string = fl.abstract_dict_to_string(fl.papers_dict[uid]['abstract'], aspect)
        publish_date = fl.papers_dict[uid]['publish_time']
        topic_words = topic_model.topics[doc[3]]
        formated_abstract = abstract_string
        
        total_occurrences = 0
    
        for word in query_text.split():
            pattern = re.compile('\b'+word+'\b', re.IGNORECASE)
            pattern.sub('<strong style="color: red">{}</strong>'.format(word), formated_abstract)
            total_occurrences += len(re.findall(word, fl.aspect_dict_to_string(fl.papers_dict[uid]['abstract'][aspect]), flags=re.IGNORECASE))
            formated_abstract = re.sub(r'\b'+word+'\b', '<strong style="color: red">{}</strong>'.format(word), formated_abstract, flags=re.IGNORECASE)
            #formated_abstract = pattern
        
        #if(total_occurrences > 0):
        final_results[uid] = {'title': fl.papers_dict[uid]['title'], 'publish_date':pandas.to_datetime(publish_date), 'abstract': formated_abstract, 'score': doc[2], 'topic': topic_words}

    #results = topic_model.search(query_text, case_sensitive=False)

    #interact(get_detailed_topics, topic_model=box.children[1])
    #for result in results:
    #    uid = fl.uids[classification][result[1]]
    #    abstract_string = fl.abstract_dict_to_string(fl.papers_dict[uid]['abstract'], aspect)
    #    publish_date = fl.papers_dict[uid]['publish_time']
        
    #    formated_abstract = abstract_string
    #    total_occurrences = 0
    #    for word in query_text.split():
            #pattern = re.compile(word, re.IGNORECASE)
            #pattern.sub('<strong style="color: red">{}</strong>'.format(word), formated_abstract)
    #        total_occurrences += len(re.findall(word, formated_abstract, flags=re.IGNORECASE))
    #        formated_abstract = re.sub(word, '<strong style="color: red">{}</strong>'.format(word), formated_abstract, flags=re.IGNORECASE)
            #formated_abstract = pattern
        
        
    #    final_results[uid] = {'title': fl.papers_dict[uid]['title'], 'publish_date':pandas.to_datetime(publish_date), 'abstract': formated_abstract, 'occurrences': total_occurrences}
            
    return final_results

df = pandas.DataFrame()


box3 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Search topic:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Textarea(
            value='',
            placeholder='Query Sentence',
            description='Query:',
            disabled=False
        ),
        widgets.Checkbox(
            value=False,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        
        widgets.RadioButtons(
            options=[('score', 'score'), ('date published', 'publish_date')],
        #    value='pineapple', # Defaults to 'pineapple'
        #    layout={'width': 'max-content'}, # If the items' names are long
            description='Sort by:',
        ),
        
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box3)
    
    query_results = get_query_results(box3.children[3].value, box3.children[1].value, box3.children[2].value)

    
    df = pandas.DataFrame.from_dict(query_results, orient='index')
    
    if(box3.children[4].value is not None):
        df = df.sort_values(by=[box3.children[4].value], ascending=False)
    
        
    display(Markdown(df.to_markdown()))
        
    #display(Markdown(pandas.DataFrame.from_dict(query_results, orient='index').to_markdown()))

    
box3.children[5].on_click(btn_eventhandler)

#display(box)

In [81]:
box4 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Search topic:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Dropdown(
            options=['transmission','incubation','environmental stability', 'risk factor', 'vaccines', 'therapeutics', 'virus genetics', 'origin', 'evolution', 'medical care', 'non-pharmaceutical interventions', 'ethical consideration', 'social science consideration',
                        'diagnostics','surveillance'],
            layout={'width': 'max-content'},
            description='Query:'
        ),
        widgets.Checkbox(
            value=False,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        
        widgets.RadioButtons(
            options=[('occurrences', 'occurrences'), ('date published', 'publish_date')],
        #    value='pineapple', # Defaults to 'pineapple'
        #    layout={'width': 'max-content'}, # If the items' names are long
            description='Sort by:',
        ),
        
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box4)
    
    query_results = get_query_results(box4.children[3].value, box4.children[1].value, box4.children[2].value)
    
    df = pandas.DataFrame.from_dict(query_results, orient='index')

    if(box4.children[4].value is not None):
        df = df.sort_values(by=[box4.children[4].value], ascending=False)
        
    display(Markdown(df.to_markdown()))

    
box4.children[5].on_click(btn_eventhandler)

In [82]:
def get_visualization(classification_bool, aspect):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    topic_model = fl.topic_models[classification][aspect]
    
    
    topic_model.visualize_documents(doc_topics=topic_model.get_doctopics(), extra_info={'title': [fl.papers_dict[fl.uids[classification][doc[1]]]['title'] for doc in topic_model.get_docs()]})
    

box5 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Visualize the documents in aspect:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Checkbox(
            value=False,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box5)
    
    query_results = get_visualization(box5.children[2].value, box5.children[1].value)

    
box5.children[3].on_click(btn_eventhandler)

In [83]:
def display_abstract(uid):
    whole_text_dict = {}
    whole_text = "<h3>"+fl.papers_dict[uid]['title']+"</h3><div class='abstract'>"
    
    for i in range(len(RESEARCH_ASPECTS)):
        for sentence in fl.papers_dict[uid]['abstract'][RESEARCH_ASPECTS[i]]:
            whole_text_dict[sentence[0]] = ""
            
            #formated_abstract = re.sub(word, '<strong style="color: red">{}</strong>'.format(word), formated_abstract, flags=re.IGNORECASE)
            
            whole_text_dict[sentence[0]] = "<span style='text-align: justify; text-justify: inter-word; background-color:"+COLORS[i]+"'> "+sentence[1]+"</span>"
            
            
            for ent in fl.papers_dict[uid]['umls_ents']:
                whole_text_dict[sentence[0]] = re.sub(ent['name'], '<div class="tooltip" style="background-color:'+COLORS[i]+' "> '+  ent['name'] + '<span class="tooltiptext">'+ent['label']+'</span></div>', whole_text_dict[sentence[0]], flags=re.IGNORECASE)
            #for word in sentence[1].split():
            #   whole_text_dict[sentence[0]] = "<span style='color:"+COLORS[i]+"'> "+sentence[1]+"</span>"
                #whole_text_dict[sentence[0]]+= "<div class='tooltip' style='color:"+COLORS[i]+"'> "+word+"<span class='tooltiptext'>Tooltip text</span></div> "
            

    for key in sorted(whole_text_dict):
        whole_text+= whole_text_dict[key]
        
    whole_text += "</div>"

    caption = ""
    for i in range(len(RESEARCH_ASPECTS)):
        caption += "<div style='display: table-row'><div style='width: 20px;display: table-cell; height:10px; background-color:"+COLORS[i]+"'></div><div style='display: table-cell'> &nbsp;"+RESEARCH_ASPECTS[i]+"</div></div>"
    
    display(Markdown(whole_text))
    display(Markdown(caption))
    

box6 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Visualize the abstract of:'),
        widgets.Dropdown(
            options=fl.papers_dict.keys(),
            layout={'width': 'max-content'},
            description='UID:'
        ),
        
        widgets.Button(
            description='Visualize'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box6)
    
    query_results = display_abstract(box6.children[1].value)

    
box6.children[2].on_click(btn_eventhandler)

# RESEARCHER INTERFACE #

### Get topic representations ###

In [84]:
display(box1)

Box(children=(Label(value='Get topics for:'), Dropdown(description='Aspect:', layout=Layout(width='max-content…

### Get documents in topic model ###

In [85]:
display(box2)

Box(children=(Label(value='Get topics for:', layout=Layout(width='max-content')), Dropdown(description='Aspect…

|          | title                                                                                                                       | publish_date        | abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |    score |
|:---------|:----------------------------------------------------------------------------------------------------------------------------|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------:|
| w5is1tbo | Vaginal vault dehiscence after robotic hysterectomy for gynecologic cancers: search for risk factors and literature review. | 2013-01-01 00:00:00 | <strong>INTRODUCTION Vaginal vault dehiscence following robotic-assisted hysterectomy for gynecologic cancer may be attributed to surgical techniques and postoperative therapeutic interventions.</strong> We searched for risk factors in patients with gynecologic cancers and complemented this with a literature review. METHODS Evaluation of prospectively gathered information on all consecutive robotic surgeries for gynecologic cancers was performed in a tertiary academic cancer center between December 2007 and March 2012. The literature was reviewed for articles relevant to "gynecologic oncology" and "robotics" with "vaginal cuff dehiscence" in the English and French languages. Respective authors were contacted to complete relevant information. RESULTS Seven dehiscences were identified of 441 cases with established gynecologic cancers. The closures in these 7 were performed using interrupted 1-Vicryl (Ethicon Inc) (3/167; 1.8%), combination of interrupted 1-Vicryl and 1-Biosyn (Covidien Inc) (3/156, 1.9%), and V-Loc (Covidien Inc) (1/118, 0.8%) sutures. Associated risk factors included adjuvant chemotherapy and/or brachytherapy, early resumption of sexual activity, and low body mass index (mean, 23 ± 3.23 kg/m²). Dehiscences occurred regardless of suturing by staff or trainees. Review of operative videos did not reveal a detectable etiologic factor, such as excessive cautery damage to the vaginal cuff or shallow tissue sutured. All 7 colporrhexis repairs were performed through a vaginal approach without the need of laparoscopy or laparotomy. CONCLUSIONS Postoperative chemotherapy, brachytherapy, and early resumption of sexual activities are risk factors for vaginal vault dehiscence. Surgical technique, particularly the use of delayed absorbable sutures, deserves further evaluation. | 0.625486 |

### Search documents based on query ###

In [86]:
display(box3)

Box(children=(Label(value='Search topic:'), Dropdown(description='Aspect:', layout=Layout(width='max-content')…

|          | title                                                                                                                                            | publish_date        | abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |     score | topic                                                                                  |
|:---------|:-------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------:|:---------------------------------------------------------------------------------------|
| zrnczve3 | Security and Fire Life Safety Threats                                                                                                            | 2013-02-15 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.319897  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| jt1ux0va | A Word of Caution in the Use of Hydroxychloroquine in the Elderly COVID-19 Population.                                                           | 2020-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.30922   | studies review data trials use clinical results using evidence quality                 |
| 17kf1q3y | Suction tube-facilitated videolaryngoscopic intubation.                                                                                          | 2015-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.298939  | studies review data trials use clinical results using evidence quality                 |
| o7gd0ubb | Covid-19: GPs call for appraisals and CQC inspections to be suspended.                                                                           | 2020-03-12 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.298939  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| ce8ng2et | COVID-19 resources.                                                                                                                              | 2020-09-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.298938  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| kcs4pav3 | This month on Twitter.                                                                                                                           | 2020-05-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.29893   | studies review data trials use clinical results using evidence quality                 |
| 95h3s7he | Excerpts from the World Medical Literature                                                                                                       | 2020-09-06 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.292276  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| ebx7ixrt | HRS/EHRA/APHRS/LAHRS/ACC/AHA worldwide practice update for telehealth and arrhythmia monitoring during and after a pandemic                      | 2020-06-11 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.290937  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 7dsyn55o | Journal Flexibility in the Troubling Times of COVID-19.                                                                                          | 2020-04-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.279381  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| pfzwsyq1 | Unexpected Benefits to Virtual Care During COVID-19.                                                                                             | 2020-06-25 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.263228  | studies review data trials use clinical results using evidence quality                 |
| fqc01jvx | Xpert MTB/Ultra assay: handle with care.                                                                                                         | 2019-11-13 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.26157   | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 7m5i1l16 | Is the content of scientific studies written on COVID-19 biostatistically satisfying?                                                            | 2020-06-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.247888  | studies review data trials use clinical results using evidence quality                 |
| el283fej | Posters of the 2020 American Association of Neuroscience Nurses Annual Meeting: The Year There Was No Wall on Which to Hang Them.                | 2020-10-01 00:00:00 | The 2020 annual meeting of the American Association of Neuroscience Nurses was canceled because of the COVID-19 pandemic. This is a summary of 27 abstracts presented in poster format that were accepted to that meeting.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.241141  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| yefaqud4 | The search for the ideal tendon repair in zone 2: strand number, anchor points and suture thickness.                                             | 2014-01-01 00:00:00 | This review article examines the mechanical factors involved in tendon repair by sutures. The repair strength, repair stiffness and gap resistance can be increased by increasing the number of core strands and anchor points, by increased anchor point efficiency and the use of peripheral sutures, and by using thicker sutures. In the future, laboratory tests could be standardized to a specific animal model and to a defined cyclic motion programme. Clinical studies support the use of multi-strand core and peripheral sutures, but two-strand core sutures are not adequate to ensure consistently good clinical results. Training surgeons in complex tendon repair techniques is essential. | 0.233197  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| wkaioy19 | Hepatobiliary & Pancreatic Diseases International (HBPD Int). Table of Contents Vol. 4, No. 1, 2005                                              | 2012-02-20 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.211651  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| pi8nkuc3 | Program Schedule                                                                                                                                 | 1983-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.205631  | studies review data trials use clinical results using evidence quality                 |
| vwcr3x71 | The Liminal Space.                                                                                                                               | 2020-06-03 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.182098  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| an3twhzi | Events calendar                                                                                                                                  | 2020-05-28 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.182091  | studies review data trials use clinical results using evidence quality                 |
| cpidk3nj | The limits of techne and episteme()                                                                                                              | 2003-12-29 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.182087  | studies review data trials use clinical results using evidence quality                 |
| anqswymg | Covid-19 and decarceration.                                                                                                                      | 2020-05-12 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.182079  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 8e8wt03q | COVID-19: Uncharted waters                                                                                                                       | 2020-04-02 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.182079  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 8rfwdu7m | The COVID-19 pand-ECMO-ic.                                                                                                                       | 2020-05-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.182079  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| oj6q71da | Top-rated MedEdPublish Articles - March 2020.                                                                                                    | 2020-07-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.175909  | studies review data trials use clinical results using evidence quality                 |
| wpjb8ay4 | Helen Salisbury: Testing times for GPs.                                                                                                          | 2020-06-02 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.17049   | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 7ubuyotj | The British Cardiovascular Society.                                                                                                              | 2013-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.166355  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 3x7p11wb | Whatever happened to universal/standard precautions?                                                                                             | 2020-04-10 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.160036  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| ishxfinx | Hard water                                                                                                                                       | 1995-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.158903  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| 5pvaeuiw | COVID-19 in Sudan: Response towards control and prevention.                                                                                      | 2020-05-01 00:00:00 | No abstract provided.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.157272  | studies review data trials use clinical results using evidence quality                 |
| tex4lbpg | The New Stability.                                                                                                                               | 2020-07-17 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.156255  | covid-19 pandemic disease coronavirus health cases care patients infection spread      |
| yeh6mmze | From Anthrax to Zika: memoirs of a disease detective                                                                                             | 2017-02-23 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.156085  | gene genetic genes strain variants biological core zika expression hbv                 |
| g5n87inp | Counting the cost of bronchiectasis.                                                                                                             | 2020-06-30 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.156085  | imaging exacerbations copd pet exacerbation stable sputum smokers dermatitis ownership |
| s5xnqa3i | The recruitment legacy of COVID-19.                                                                                                              | 2020-08-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.149489  | studies review data trials use clinical results using evidence quality                 |
| l46stjow | Inhaltsverzeichnis                                                                                                                               | 2020-06-10 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| eu7ryw7x | Bericht aus dem Vorstand                                                                                                                         | 2020-06-08 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| ehw61gu6 |                                                                                                                                                  | 2020-05-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| 7qxh3art | Respiratorische Erkrankungen                                                                                                                     | 2007-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| hsp5goxp | Acronyms and Abbreviations                                                                                                                       | 2007-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| 2piml3jl | Marian Horzinek, Obituary                                                                                                                        | 2016-09-14 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| c8jzexd4 | Who's boss                                                                                                                                       | 2006-11-18 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| rr4yep41 | Bevölkerung                                                                                                                                      | 2005-01-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| 1gf8n3lf | Posters                                                                                                                                          | 2004-11-04 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| elotmmf4 | Antimikrobielle Therapie                                                                                                                         | 2009-03-03 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| yk3fp4qo | [Systematic perinatal management of the pregnant women and neonates during the epidemic of COVID-19].                                            | 2020-03-25 00:00:00 | 2019新型冠状病毒感染流行，孕产妇作为一类特殊的易感人群，围产期的任何时段都可能受到感染。感染后往往病情进展较快较重，严重威胁母儿健康。因此，如何加强覆盖妊娠期、分娩期和产褥期的围产期保健全程防控，以减少不良母儿结局、保障母儿安全就显得十分重要。本文就围产期的全程防控提出了有针对性的建议，以供参考。.                                                                                                                                                                                                                                                                                                                                                                                                   | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| 1a4xj295 | Társasági hírek.                                                                                                                                 | 2020-06-01 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| otpjh7s2 | Gastrointestinale Infektiologie                                                                                                                  | 2020-04-24 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| z02wmrjh | Dartelen                                                                                                                                         | 2020-05-15 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| ll7e0gua | Liebe Leserin, lieber Leser,                                                                                                                     | 2020-05-12 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| 01uxvjvz | Atemwegsinfekte                                                                                                                                  | 2020-05-29 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| 8tjnrp0w | Inhaltsverzeichnis                                                                                                                               | 2020-06-05 00:00:00 |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |
| o34afjmq | [Clinical observation on increasing the positive rate of novel coronavirus nucleic acid tests by sputum excretion induced by nebulizer therapy]. | 2020-04-12 00:00:00 | 对多次咽拭子病毒核酸检测阴性的2例新型冠状病毒肺炎疑似患者，用3%的高渗盐水进行压力雾化诱导排痰获取呼吸道深部痰液，检测痰液中新型冠状病毒核酸。结果显示，使用3%的高渗盐水雾化诱导排痰并对痰液中新型冠状病毒进行核酸检测可明显提高检测的阳性率，可提高疑似患者的确诊率，缩短确诊时间。.                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.0185185 | vaccine protein immunity reactivity malaria study domains antibody codon severe        |

### Search documents based on pre-defined query ###

In [75]:
display(box4)

Box(children=(Label(value='Search topic:'), Dropdown(description='Aspect:', layout=Layout(width='max-content')…

### Visualize aspect documents ###

In [76]:
display(box5)

Box(children=(Label(value='Visualize the documents in aspect:'), Dropdown(description='Aspect:', layout=Layout…

In [77]:
#fl.topic_models['all_papers']['Background']
fl.papers_dict
#fl.uids['covid_papers']

{'5yhe786e': {'title': 'Debate: Transfusing to normal haemoglobin levels will not improve outcome',
  'abstract': {'Background': [(0,
     'Recent evidence suggests that critically ill patients are able to tolerate lower levels of haemoglobin than was previously believed.'),
    (2,
     'Although many questions remain, many laboratory and clinical studies, including a recent randomized controlled trial (RCT), have established that transfusing to normal haemoglobin concentrations does not improve organ failure and mortality in the critically ill patient.')],
   'Purpose': [(1,
     'It is our goal to show that transfusing to a level of 100 g/l does not improve mortality and other clinically important outcomes in a critical care setting.')],
   'Method': [],
   'Finding_Contribution': [(3,
     'In addition, a restrictive transfusion strategy will reduce exposure to allogeneic transfusions, result in more efficient use of red blood cells (RBCs), save blood overall, and decrease health c

### Visualize abstract ###

In [78]:
display(box6)

Box(children=(Label(value='Visualize the abstract of:'), Dropdown(description='UID:', layout=Layout(width='max…